In [1]:
#会员活动数据营销预测
import time
import numpy as np
import pandas as pd
#导入数据处理库
from sklearn.preprocessing import OneHotEncoder
#导入模型选择库
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
#导入特征选择库
from sklearn.feature_selection import SelectPercentile,f_classif
#导入集成算法库
from sklearn.ensemble import AdaBoostClassifier
#导入Pipeline库
from sklearn.pipeline import Pipeline
#导入评估指标库
from sklearn.metrics import accuracy_score

In [2]:
#加载数据集
df=pd.read_excel('market_activity.xlsx')

In [3]:
#基本状态查看
print('数据概览:')
print('-'*80)
#查看数据集形状
print('数据记录数:{0}\t特征数量:{1}'.format(df.shape[0],(df.shape[1]-1)))
#查看前5条数据
print(df.head(5))
print('-'*80)
print('基本描述性信息:')
#查看数据基本描述性信息
print(df.describe())
print('-'*80)
#查看数据类型
print('数据类型:')
print(df.dtypes)

数据概览:
--------------------------------------------------------------------------------
数据记录数:48000	特征数量:13
    age  total_pageviews  edu  edu_ages  user_level  industry  value_level  \
0  24.0          32921.0  2.0       9.0         1.0       6.0            1   
1  26.0         397317.0  3.0      14.0         1.0       4.0            1   
2  19.0         170653.0  2.0       9.0         1.0      12.0            4   
3  51.0         259323.0  1.0       NaN         2.0       2.0            2   
4  42.0         254817.0  2.0      10.0         1.0       4.0            1   

   act_level  sex  blue_money  red_money  work_hours  region  response  
0        1.0  1.0           0        0.0          40     1.0         0  
1        1.0  0.0           0     1876.0          40     1.0         0  
2        1.0  1.0           0        0.0          40    15.0         0  
3        1.0  1.0           0        0.0          50     1.0         1  
4        1.0  0.0           0     1340.0          40     1.

In [4]:
#缺失值审查
#判断每一列是否具有缺失值
na_cols=df.isnull().any(axis=0)
print('具有缺失值的列:')
#查看具有缺失值的列
print(na_cols)  
print('-'*30)
#判断每一行是否具有缺失值
na_lines=df.isnull().any(axis=1)
#查看具有缺失值的行总记录数
print('具有缺失值的行总记录数:{0}'.format(na_lines.sum()))

具有缺失值的列:
age                 True
total_pageviews     True
edu                 True
edu_ages            True
user_level          True
industry            True
value_level        False
act_level           True
sex                 True
blue_money         False
red_money           True
work_hours         False
region              True
response           False
dtype: bool
------------------------------
具有缺失值的行总记录数:17


In [5]:
#类样本均衡审查
#以response为分类汇总维度计数统计
print('以目标标签为分类汇总计数统计:')
print(df.groupby(df['response']).count())

以目标标签为分类汇总计数统计:
            age  total_pageviews    edu  edu_ages  user_level  industry  \
response                                                                  
0         36503            36503  36504     36504       36503     36503   
1         11496            11496  11495     11495       11496     11495   

          value_level  act_level    sex  blue_money  red_money  work_hours  \
response                                                                     
0               36504      36503  36503       36504      36504       36504   
1               11496      11496  11496       11496      11495       11496   

          region  
response          
0          36497  
1          11496  


In [6]:
#缺失值替换
#定义各个列数据转换方法
na_rules={'age':df['age'].mean(),
          'total_pageviews':df['total_pageviews'].mean(),
          'edu':df['edu'].median(),
          'edu_ages':df['edu_ages'].median(),
          'user_level':df['user_level'].median(),
          'industry':df['user_level'].median(),
          'act_level':df['act_level'].median(),
          'sex':df['sex'].median(),
          'red_money':df['red_money'].mean(),
          'region':df['region'].median()
         }
#使用指定方法填充缺失值
df1=df.fillna(na_rules)
#查找是否还有缺失值
print('查找是否还有缺失值:')
print(df1.isnull().any().sum())

查找是否还有缺失值:
0


In [7]:
#分类与顺序变量类型转换
df2=df1
#定义要转换的列及其数据类型
var_list={'edu':'int32',
          'user_level':'int32',
          'industry':'int32',
          'value_level':'int32',
          'act_level':'int32',
          'sex':'int32',
          'region':'int32'
         }
#循环读出列名和对应的数据类型
for var,type in var_list.iteritems():
    #数据类型转换
    df2[var]=df1[var].astype(type) 
print('转换后的数据类型:')
print(df2.dtypes)

转换后的数据类型:
age                float64
total_pageviews    float64
edu                  int32
edu_ages           float64
user_level           int32
industry             int32
value_level          int32
act_level            int32
sex                  int32
blue_money           int64
red_money          float64
work_hours           int64
region               int32
response             int64
dtype: object


In [8]:
#将分类与顺序变量转换为二值化标志变量
#选择要做标志转换的列名
convert_cols=['edu','user_level','industry','value_level','act_level','sex','region']
#选择要做标志转换的数据
df_con=df2[convert_cols]
#选择不作标志转换的列
df_org=df2[['age','total_pageviews','edu_ages', 'blue_money','red_money','work_hours','response']].values
#建立标志转换模型对象
enc=OneHotEncoder(categories='auto')
#训练模型
enc.fit(df_con)
#转换数据并输出为数组格式
df_con_new=enc.transform(df_con).toarray()
#将未转换的数据与转换后的数据合并
dataset=np.hstack((df_con_new, df_org))

In [9]:
#划分经过预处理后的数据集
df_new=pd.DataFrame(dataset)
#获得样本特征集与目标结果集
x,y=df_new.iloc[:,:dataset.shape[1]-1],df_new.iloc[:,-1]
#将样本特征集与目标结果集随机划分为训练集和测试集，测试集占比20%
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [10]:
#获得最佳模型参数
#np.seterr(divide='ignore',invalid='ignore')
np.seterr(all="ignore")
#使用f_classif方法选择特征最明显的50%数量的特征
transform=SelectPercentile(f_classif,percentile=50)
#建立AdaBoostClassifier模型对象
model_adaboost=AdaBoostClassifier()
#建立由特征选择和分类模型构成的管道对象
model_pipe=Pipeline(steps=[('ANOVA',transform),('model_adaboost',model_adaboost)])
#设置交叉检验次数
cv=StratifiedKFold(5)
#设置分类器参数列表
n_estimators=[20,40,60,80,100]
#设置交叉检验指标
score_methods=['accuracy','f1','precision','recall','roc_auc']
#建立空列表用于存放不同参数方法、交叉检验评估指标的均值列表
mean_list=list()
#建立空列表用于存放不同参数方法、交叉检验评估指标的标准差列表
std_list=list()
#循环读出每个参数值
for parameter in n_estimators:
    #记录训练开始的时间
    t1=time.time()
    #建立空列表用于存放不同交叉检验下各个评估指标的详细数据
    score_list=list()
    #查看当前模型使用的分类器个数
    print ('当前模型使用的分类器个数: %s' % parameter)
    #循环读出每个交叉检验指标
    for score_method in score_methods:
        #通过管道设置分类器个数
        model_pipe.set_params(model_adaboost__n_estimators=parameter)
        #使用交叉检验计算指定指标的得分
        score_tmp=cross_val_score(model_pipe,x_train,y_train,scoring=score_method,cv=cv)
        #将交叉检验得分存储到列表
        score_list.append(score_tmp)
    #将交叉检验详细数据转换为矩阵
    score_matrix=pd.DataFrame(np.array(score_list), index=score_methods)
    #计算每个评估指标的均值
    score_mean=score_matrix.mean(axis=1).rename('mean')
    #计算每个评估指标的标准差
    score_std=score_matrix.std(axis=1).rename('std')
    #将原始详细数据和均值、标准差合并
    score_pd=pd.concat([score_matrix, score_mean, score_std], axis=1)
    #将每个参数得到的各指标均值追加到列表
    mean_list.append(score_mean)
    #将每个参数得到的各指标标准差追加到列表
    std_list.append(score_std)
    #查看每个参数得到的交叉检验指标数据，只保留2位小数
    print(score_pd.round(2))
    #计算每个参数下算法用时
    t2=time.time()
    #计算时间间隔
    tt=t2-t1
    print('运行时间: %s' % str(tt))
    print('-'*60)
#建立所有分类器参数得到的交叉检验的均值矩阵
mean_matrix=np.array(mean_list).T
#建立所有分类器参数得到的交叉检验的标准差矩阵
std_matrix=np.array(std_list).T
#将均值矩阵转换为数据框
mean_pd=pd.DataFrame(mean_matrix,index=score_methods,columns=n_estimators)
#将均值标准差转换为数据框
std_pd=pd.DataFrame(std_matrix,index=score_methods,columns=n_estimators)
print('均值矩阵:')
print(mean_pd)
print('-'*60)
print('标准差矩阵:')
print(std_pd)

当前模型使用的分类器个数: 20


D:\devsetup\anaconda2\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: UserWarning: Features [85] are constant.
  UserWarning)


              0     1     2     3     4  mean   std
accuracy   0.85  0.86  0.86  0.85  0.85  0.85  0.00
f1         0.65  0.67  0.66  0.65  0.64  0.65  0.01
precision  0.76  0.76  0.77  0.74  0.75  0.76  0.01
recall     0.56  0.61  0.57  0.57  0.56  0.58  0.02
roc_auc    0.91  0.91  0.91  0.91  0.91  0.91  0.00
运行时间: 31.0020000935
------------------------------------------------------------
当前模型使用的分类器个数: 40
              0     1     2     3     4  mean   std
accuracy   0.86  0.87  0.86  0.85  0.85  0.86  0.00
f1         0.67  0.69  0.67  0.66  0.66  0.67  0.01
precision  0.75  0.77  0.77  0.75  0.75  0.76  0.01
recall     0.61  0.62  0.59  0.58  0.58  0.60  0.02
roc_auc    0.91  0.92  0.92  0.91  0.91  0.91  0.00
运行时间: 51.5839998722
------------------------------------------------------------
当前模型使用的分类器个数: 60
              0     1     2     3     4  mean   std
accuracy   0.86  0.87  0.86  0.86  0.86  0.86  0.00
f1         0.67  0.70  0.68  0.66  0.66  0.68  0.01
precision  0.75  0.77  0

In [11]:
#分类模型训练
#应用特征选择对象选择要参与建模的特征变量
transform.fit(x_train,y_train)
#获得具有显著性特征的特征变量
x_final=transform.transform(x_train)
#从均值与标准差矩阵信息可以看出分类器数为80的结果与100基本一致，由于分类器数为100的提升效果不明显，所以模型的最佳分类器数设置为80
final_model=AdaBoostClassifier(n_estimators=80,random_state=0)
#训练模型
final_model.fit(x_final,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=80, random_state=0)

In [12]:
#输出预测值以及预测概率
x_test_final=transform.transform(x_test)
#获得预测标签
predict_labels=pd.DataFrame(final_model.predict(x_test_final),columns=['pred_labels'],dtype=np.int64)
predict_labels=pd.DataFrame(predict_labels.values.tolist(),index=x_test.index,columns=predict_labels.columns.values.tolist())
#获得预测概率
predict_labels_pro=pd.DataFrame(final_model.predict_proba(x_test_final),columns=['pro1','pro2'])
predict_labels_pro=pd.DataFrame(predict_labels_pro.values.tolist(),index=x_test.index,columns=predict_labels_pro.columns.values.tolist())
#将测试集、预测标签和预测概率合并
predict_pd=pd.concat([df.iloc[x_test.index],predict_labels,predict_labels_pro.round(4)],axis=1)
print('测试集与预测信息:')
print(predict_pd[['value_level','act_level','response','pred_labels','pro1','pro2']].head(5))

测试集与预测信息:
       value_level  act_level  response  pred_labels    pro1    pro2
37957            3        1.0         1            1  0.4978  0.5022
45965            2        1.0         0            1  0.4957  0.5043
33541            5        3.0         0            0  0.5117  0.4883
9905             2        1.0         1            0  0.5013  0.4987
9472             4        1.0         0            0  0.5212  0.4788


In [13]:
#评估模型
print ('模型准确率为:{0}'.format(accuracy_score(y_test, predict_labels)))

模型准确率为:0.860520833333
